In [35]:
import spacy
from spacy.tokens import DocBin
from spacy.training import Example
from spacy.util import Config
from spacy.training import Corpus
import random
from tqdm import tqdm
import pandas as pd 
import os
import subprocess
import ast

print("all libraries loaded")

all libraries loaded


In [36]:
print("loading spacy model...")
nlp_fr = spacy.load("fr_core_news_sm")
print("spaCy model 'fr_core_news_sm' charge avec succes !!!")

loading spacy model...
spaCy model 'fr_core_news_sm' charge avec succes !!!


In [37]:
texte_simple = '''Je voudrais aller de Ermont à Sannois.'''
doc = nlp_fr(texte_simple)
spacy.displacy.render(doc, style="ent", jupyter=True)

In [38]:
# Charger le fichier CSV
df = pd.read_csv('../../dataset/raw/initial_training_data.csv')
print("\nfichier csv charge avec succes\n")


fichier csv charge avec succes



In [39]:
# premieres lignes du dataset pour verifier le chargement
df.head(20)

,Phrase,Entities
0,Je voudrais aller de La Ciotat à Sarreguemines.,"[[21, 30, 'DEPARTURE'], [33, 46, 'DESTINATION']]"
1,Comment me rendre à port-boulet depuis la gare...,"[[50, 56, 'DEPARTURE']]"
2,Je veux aller voir mon ami albert à Villeneuve...,"[[36, 54, 'DEPARTURE'], [69, 74, 'DESTINATION']]"
3,Y a-t-il des trains de Nancy à montaigu ?,"[[23, 28, 'DEPARTURE']]"
4,Une phrase sans origine ni destination.,[]
5,"Si pas de numéro de séquence, on considère que...",[]
6,Is there any train going from Corbeil-Essonnes...,"[[30, 46, 'DEPARTURE'], [50, 58, 'DESTINATION']]"
7,Je souhaite me rendre à Vernon en partant de M...,"[[24, 30, 'DEPARTURE'], [45, 54, 'DESTINATION']]"
8,Je quitte Cournon-D'Auvergne pour aller à Dax,"[[10, 28, 'DEPARTURE'], [42, 45, 'DESTINATION']]"
9,"En partant de Alençon, je veux me rendre à La ...","[[14, 21, 'DEPARTURE'], [43, 55, 'DESTINATION']]"


In [40]:
# Limiter à 400 lignes pour accélérer l'entraînement
df = df.sample(n=400, random_state=42)
print("\ndatabase limite a 400 lignes pour l'entrainement\n")


database limite a 400 lignes pour l'entrainement



In [41]:
# creation d'un conteneur pour stocker les documents au format spaCy
db = DocBin()
print("\nconteneur DocBin cree avec succes\n")


conteneur DocBin cree avec succes



In [42]:
# on melange les donnees pour eviter les biais dans l'apprentissage
df = df.sample(frac=1).reset_index(drop=True)
print("\ndonnees melangees avec succes\n")


donnees melangees avec succes



In [43]:
# on convertit les donnes au format spacy
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    text = row['Phrase']
    doc = nlp_fr.make_doc(text) 
    ents = []

    entities = ast.literal_eval(row['Entities'])

    for entity in entities:
        start, end, label = entity
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is not None:
            ents.append(span)

    doc.ents = ents 
    db.add(doc)
    
print("\nles données ont été bien converties au format json\n")

100%|██████████| 400/400 [00:00<00:00, 968.97it/s] 


les données ont été bien converties au format json



In [44]:
# on enregistre les donnees converties au format spaCy
db.to_disk("../../dataset/processed/processed_datas.spacy")
print("\ndonnees converties et sauvegardees au format spaCy\n")


donnees converties et sauvegardees au format spaCy



In [45]:
# ajout du composant NER au modele actuel
ner = nlp_fr.get_pipe("ner")

In [46]:
# Ajouter de nouveaux labels d'entités à partir des données
for _, row in df.iterrows():
    entities = ast.literal_eval(row['Entities'])
    for entity in entities:
        _, _, label = entity
        ner.add_label(label)
print("\nles nouveaux labels NER ont été ajoutées au modèle\n")


les nouveaux labels NER ont été ajoutées au modèle



In [47]:
# on cree le fichier de configuration
os.system('python -m spacy init config ./configuration.cfg --lang fr --pipeline ner --optimize efficiency --force')
print("\nfichier de configuration genere avec succes\n")


fichier de configuration genere avec succes



In [48]:
# entrainement avec un modele leger et config optimisee
command = 'python -m spacy train ./configuration.cfg --output ../../models/saved_models --paths.train ../../dataset/processed/processed_datas.spacy --paths.dev ../../dataset/processed/processed_datas.spacy'

with tqdm(total=100, desc="Entrainement du modele") as pbar:
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in process.stdout:
        print(line.decode(), end='')
        pbar.update(1)

process.wait()
print("\nentrainement du modele termine avec succes\n")

Entrainement du modele:   1%|          | 1/100 [00:08<14:10,  8.59s/it]

ℹ Saving to output directory: ..\..\models\saved_models
ℹ Using CPU

=========================== Initializing pipeline ===========================


Entrainement du modele:   5%|▌         | 5/100 [00:17<05:03,  3.19s/it]

✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------


Entrainement du modele:  12%|█▏        | 12/100 [00:18<01:37,  1.11s/it]

...

208    2600          0.00      0.00  100.00  100.00  100.00    1.00


Entrainement du modele:  27%|██▋       | 27/100 [10:00<49:24, 40.61s/it]  

241    2800         55.63      7.89  100.00  100.00  100.00    1.00
✔ Saved pipeline to output directory
..\..\models\saved_models\model-last


Entrainement du modele:  28%|██▊       | 28/100 [10:00<25:45, 21.46s/it]


entrainement du modele termine avec succes



In [49]:
# on charge le modele entraîne
nlp_itineraire = spacy.load("../../models/saved_models/model-best")
print("modele entraine charge avec succes")

modele entraine charge avec succes


In [50]:
# Tester le modèle avec un texte plus complexe
texte_test = '''
Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence.
Je compte me rendre à Bordeaux depuis Marseille pour rendre visite à ma soeur.
Je dois regarder les trains Toulouse - Brest pour aller voir mon ami Albert.
Je dois planifier un voyage Nice Toulouse pour les prochaines vacances.
Une réunion de travail m'oblige à faire Paris - Clermont-Ferrand la semaine prochaine.
'''

# Use the model to identify entities in the text
doc_test = nlp_itineraire(texte_test)

# Display the recognized entities
spacy.displacy.render(doc_test, style="ent", jupyter=True)